In [1]:
import sys
sys.path.append('/data2/junhong/proj/text_guide_attack')
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
import torch
import pyarrow.parquet as pq
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import Dataset, DataLoader
from dataset import MS_COCO, collate_fn
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import argparse
from tqdm import tqdm
from model.clip_unet import CLIP_encoder_decoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def args(argparse):
    argparse.add_argument("--train", type=str, default=True)
    argparse.add_argument("--clip_model_path", type=str, default="/data2/ModelWarehouse/clip-vit-base-patch32")
    argparse.add_argument("--image_path", type=str, default="/data2/zhiyu/data/coco/images/train2017")
    argparse.add_argument("--data_path", type=str,
                      default="/data2/junhong/proj/text_guide_attack/data/mscoco_exist.parquet")
    argparse.add_argument("--epoch", type=int, default=20)
    argparse.add_argument("--batch_size", type=int, default=256)
    argparse.add_argument("--shuffle", type=bool, default=True)
    argparse.add_argument("--mode", type=str, default="test")
    argparse.add_argument("--model_path", type=str, default="save_model/model_epoch_20.pth")
    args = argparse.parse_args()
    return args

In [2]:
# argparse= argparse.ArgumentParser()
# args= args(argparse)
processor = CLIPProcessor.from_pretrained("/data2/ModelWarehouse/clip-vit-base-patch32")
encoder = CLIPModel.from_pretrained("/data2/ModelWarehouse/clip-vit-base-patch32").to(device).eval()

In [3]:
class MS_COCO(Dataset):
    def __init__(self, data_path,image_path):
        self.args = args
        self.data_list=pq.read_table(data_path)
        self.data_list=self.data_list.to_pandas()
        self.image_path=image_path

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):

        text = self.data_list['TEXT'][idx]

        image_name = self.data_list['URL'][idx]

        img_path=os.path.join(self.image_path,image_name)
        img = Image.open(img_path).convert('RGB')

        # You can perform additional transformations on the image here if needed

        return img, text
batch_size = 256
data_path="/data2/junhong/proj/text_guide_attack/data/mscoco_exist.parquet"
image_path="/data2/zhiyu/data/coco/images/train2017"
train_dataset = MS_COCO(data_path,image_path)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,collate_fn=collate_fn)

In [4]:
def process_text(texts):
    input_text = []
    for text in texts:
        input_text.append(text[0])
    return input_text
for (images, texts) in tqdm(train_data_loader, desc="Testing", position=0):
    inputs = processor(text=process_text(texts), images=images, return_tensors="pt", padding=True).to(device)
    break

Testing:   0%|                                          | 0/463 [00:03<?, ?it/s]


In [5]:
encoder(**inputs)

CLIPOutput(loss=None, logits_per_image=tensor([[28.3239, 14.1571, 18.9245,  ..., 11.2521,  8.8391, 13.8871],
        [18.4652, 30.3185,  9.9260,  ..., 12.1234, 18.7343, 12.8444],
        [17.2572, 12.5573, 28.3465,  ..., 11.9973, 10.2950, 11.0302],
        ...,
        [18.9985, 12.8682,  8.7138,  ..., 33.1035, 11.9111, 13.3936],
        [17.2938, 14.5991, 14.4264,  ..., 10.5532, 37.3439,  9.2228],
        [16.4232, 15.4139, 13.6517,  ..., 14.3283, 10.8179, 31.8879]],
       device='cuda:0', grad_fn=<TBackward0>), logits_per_text=tensor([[28.3239, 18.4652, 17.2572,  ..., 18.9985, 17.2938, 16.4232],
        [14.1571, 30.3185, 12.5573,  ..., 12.8682, 14.5991, 15.4139],
        [18.9245,  9.9260, 28.3465,  ...,  8.7138, 14.4264, 13.6517],
        ...,
        [11.2521, 12.1234, 11.9973,  ..., 33.1035, 10.5532, 14.3283],
        [ 8.8391, 18.7343, 10.2950,  ..., 11.9111, 37.3439, 10.8179],
        [13.8871, 12.8444, 11.0302,  ..., 13.3936,  9.2228, 31.8879]],
       device='cuda:0', grad_f

In [6]:
def process_text(texts):
    input_text = []
    for text in texts:
        input_text.append(text[0])
    return input_text

def calculate_cos(image_emb, output_encode):
    cos_sim = torch.nn.functional.cosine_similarity(image_emb, output_encode, dim=1)
    return cos_sim.sum()
sim=0.0
eval_batch=30
count=0
with torch.no_grad():
    for (images, texts) in tqdm(train_data_loader, desc="Testing", position=0):
        # images=images.to(device)
        # texts=texts.to(device)
        inputs = processor(text=process_text(texts), images=images, return_tensors="pt", padding=True).to(device)
        # print(type(inputs))
        output=encoder(**inputs)
        img_embeds=output.image_embeds
        text_embeds=output.text_embeds
        sim+=calculate_cos(img_embeds,text_embeds)
        count+=1
        print(f"count_{count}:",sim/(count*batch_size))
        if count==eval_batch:
            break
    # print("sim:",sim/(eval_batch*batch_size))




Testing:   0%|                                  | 1/463 [00:03<28:37,  3.72s/it]

count_1: tensor(0.3039, device='cuda:0')


Testing:   0%|▏                                 | 2/463 [00:07<29:26,  3.83s/it]

count_2: tensor(0.3043, device='cuda:0')


Testing:   1%|▏                                 | 3/463 [00:11<29:09,  3.80s/it]

...

count_28: tensor(0.3038, device='cuda:0')


Testing:   6%|██                               | 29/463 [01:46<26:41,  3.69s/it]

count_29: tensor(0.3038, device='cuda:0')


Testing:   6%|██                               | 29/463 [01:49<27:25,  3.79s/it]

count_30: tensor(0.3038, device='cuda:0')
